# DB2-Salesforce connector: Basic user information updates

%load_ext autoreload
%autoreload 2

In [4]:
# Parameters
hours_range = 24*365*1 # number of hours to look back
#change this back to one year, once you're done

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


In [5]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [16]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range))\
                    .strftime('%Y-%m-%d')
lvd_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=24*5))\
                    .strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '{}' or lastvisitDate >= '{}'".format(date_cutoff,lvd_cutoff)

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '2019-11-24'


In [17]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key \
in ('orgtype', 'organization', 'orcid')", engine)

In [18]:
# display
display(df.head(2))

display(profile_df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,272150,Xing He,-125427,0,-125427@invalid,-1,2019-11-24 00:01:07,2019-11-24 00:01:08
1,272151,LUISA BRANDI BECERRA,lbrandi,0,lbrandi_b@hotmail.com,1,2019-11-24 01:56:16,2019-11-24 02:03:00


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5
1,2,10060,orgtype,university,2,5


In [19]:
display(sorted(df['lastvisitDate'].to_list())[-5:])

[Timestamp('2020-10-23 02:10:01'),
 Timestamp('2020-10-23 02:20:33'),
 Timestamp('2020-10-23 02:31:21'),
 Timestamp('2020-10-23 02:49:08'),
 Timestamp('2020-10-23 03:06:30')]

In [6]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})
display(df.head(1))

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN


In [7]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [8]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [9]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_host'] = df['email'].apply(get_domain)

In [10]:
# save a constuct for debugging
from copy import deepcopy
temp = deepcopy(df)

In [11]:
# get domain by nanoHUB organization field
df = deepcopy(temp)
display(df.head(2))
df['organization'] = df['organization'].str.lower()
df = df.fillna('   ')

df.loc[(df['organization']=='nan'),'organization'] = None
df.loc[(df['organization']=='NaN'),'organization'] = None

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN,gmail.com
1,1686,nanoHUB nanoHUB,nanohub,0,apps@nanohub.org,0,2014-06-26 19:38:57,NaT,NaN,Purdue University,NaN,nanohub.org


display(df.tail())

In [12]:
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

In [13]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [14]:
display(domain_df.head(2))

,domain_name,domain,industry,size,country
0,strategic interns,strategicinterns.com,education management,1-10,india
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain


In [15]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)
display(domain_df.head(1))
# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set
0,strategic interns,strategicinterns.com,education management,1-10,india,"{strategic, interns}"


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,,,,,gmail.com,{}


In [16]:
# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [17]:
display(domain_df.head(1))
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
0,strategic interns,strategicinterns.com,education management,1-10,india,"{strategic, interns}",interns-strategic


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,,,,,gmail.com,{},None


## Collaborative filtering for commerical email hosts

In [18]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts() #returns uniques in descending order
    
    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False
    
    else:
        return False
    
    
is_email_commercial = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [19]:
temp2 = deepcopy(df)

In [20]:
df = deepcopy(temp2)

In [21]:
# mark commerical email addresses
df = pd.merge(df, is_email_commercial.reset_index(name='is_email_commercial'),\
         how='left', left_on='email_host', right_on='email_host' ,)

In [22]:
display(df.tail(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial
185544,302613,Brent Noesges,noesges.1,0,noesges.1@osu.edu,0,2020-10-08 19:49:47,2020-10-08 19:49:47,universitygraduate,,,osu.edu,{},None,False


In [23]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commercial[is_email_commercial].index)]['domain'],\
         how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,998,1686
name,hubrepo hubrepo,nanoHUB nanoHUB
username,hubrepo,nanohub
block,0,0
email,nkissebe@gmail.com,apps@nanohub.org
sendEmail,0,0
registerDate,2014-11-13 21:09:09,2014-06-26 19:38:57
lastvisitDate,,
orgtype,,
organization,,purdue university


## Fuzzy derive organization from nanoHUB profiles

In [24]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
# only take values that exist
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),\
         how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash')\
        .drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

,160542,109780,39476,181731,10102
id,270363,181927,90043,297431,57777
name,Ahmed gamal gamal,ANIKET ARJUN MANKUMRE,Ryo Igarashi,Benjamin Leon Gutman,Takeshi Iwaa
username,ahmedgsoliman,mankumre96,rigarash,bgutman,tiwa
block,0,0,0,0,0
email,AhmedGamal_Soliman@agr.asu.edu.eg,aniketmankumre96@gmail.com,rigarash@gmail.com,bgutman@purdue.edu,tak.iws@gmail.com
sendEmail,1,1,-1,0,0
registerDate,2019-11-03 20:55:55,2017-10-13 05:11:31,2013-10-22 02:37:20,2020-08-21 17:57:08,2011-09-02 01:19:55
lastvisitDate,2020-02-06 00:52:43,2018-01-23 09:19:41,2013-10-22 03:14:11,2020-10-08 15:00:46,2011-09-14 05:41:20
orgtype,universityundergraduate,,,universitygraduate,government
organization,,,,purdue university,


In [25]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))]\
                    .org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [26]:
display(domain_subset_df.head(2))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain,"{bosque, comunidad}",bosque-comunidad
4,"cardinal strategies, llc",cardinal-strategies.com,civil engineering,11-50,united states,"{cardinal, strategies, llc}",cardinal-llc-strategies


In [27]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0] 


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [28]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer')\
                      , how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [29]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile
    
    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email    
    
    # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer
    
    # else, no organization can be concluded, return none
    return None

    
derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('')\
                                                        .apply(get_composite_domain, axis=1)

In [30]:
display(derived_df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,,,,,gmail.com,{},None,True,NaN,NaN,NaN,None
1,1686,nanoHUB nanoHUB,nanohub,0,apps@nanohub.org,0,2014-06-26 19:38:57,,,purdue university,,nanohub.org,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu


## Update Salesforce institution 

In [31]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain') 
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,texas a&m university,tamu.edu,higher education,10001+,united states


In [32]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,purdue university,texas a&m university
Size_Range__c,10001+,10001+
Country__c,united states,united states
Domain__c,purdue.edu,tamu.edu
Industry__c,higher education,higher education


In [33]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

Obtained Salesforce access token ...... True


In [34]:
display(df_sf.head(2))
display(df_sf.tail(2))


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,texas a&m university,10001+,united states,tamu.edu,higher education


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
2851,airy3d,11-50,canada,airy3d.com,consumer electronics
2852,universitat de lleida,1001-5000,spain,udl.cat,higher education


In [35]:
## correcting name overflows
names = df_sf['Name'].str.len()

sf_char_limit = 80
name_indexes = names.index[names >= sf_char_limit].tolist()
for index in name_indexes:
    df_sf['Name'][index] =  df_sf['Name'][index][0:sf_char_limit]

In [36]:
display(df_sf.head(2))

,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,texas a&m university,10001+,united states,tamu.edu,higher education


In [37]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000QhXGHAA3
hello
[Success] CSV upload successful. Job ID = 7505w00000QhXGHAA3
[Success] Closing job successful. Job ID = 7505w00000QhXGHAA3


In [38]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000QhXGHAA3',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-10-08T20:06:39.000+0000',
 'systemModstamp': '2020-10-08T20:06:43.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [39]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Obtain newly updated organization IDs from Salesforce

In [40]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [41]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000QhXGMAA3
{"id":"7505w00000QhXGMAA3","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-10-08T20:06:44.000+0000","systemModstamp":"2020-10-08T20:06:45.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000QhXGMAA3","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-10-08T20:06:44.000+0000","systemModstamp":"2020-10-08T20:06:45.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6297,"retries":0,"totalProcessingTime":606}
[Success] Bulk job completed successfully.


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [42]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final')\
            .drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_org_ID'})

In [43]:
# display
display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,48909,Nader Engheta,nader_engheta_1,0,engheta@ee.upenn.edu,0,2010-11-03 02:42:28,2010-11-03 02:50:16,universityfaculty,university of pennsylvania,,ee.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu
1,a0r5w00000V42c0AAB,58099,Brian Cohen,bscohen,0,bscohen@seas.upenn.edu,1,2011-09-09 20:51:43,,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu


## Match data with Salesforce format

In [44]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [45]:
## stop here
display(df.head(1))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,48909,Nader Engheta,nader_engheta_1,0,engheta@ee.upenn.edu,0,2010-11-03 02:42:28,2010-11-03 02:50:16,universityfaculty,university of pennsylvania,,ee.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu


In [46]:
## if last visit date is empty - set it equal to the register date
lvd = df.lastvisitDate.index[df.lastvisitDate == '   '].tolist()

for i in lvd:
    df.iloc[i,8] = df.iloc[i,7]

display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,48909,Nader Engheta,nader_engheta_1,0,engheta@ee.upenn.edu,0,2010-11-03 02:42:28,2010-11-03 02:50:16,universityfaculty,university of pennsylvania,,ee.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu
1,a0r5w00000V42c0AAB,58099,Brian Cohen,bscohen,0,bscohen@seas.upenn.edu,1,2011-09-09 20:51:43,2011-09-09 20:51:43,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu


In [47]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID']
df_sf['Organization_composite__c'] = df['Salesforce_org_ID']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,Nader,Brian
Middle_name__c,,
lastname,Engheta,Cohen
nanoHUB_user_ID__c,48909,58099
nanoHUB_username__c,nader_engheta_1,bscohen
Email,engheta@ee.upenn.edu,bscohen@seas.upenn.edu
HasOptedOutOfEmail,1,0
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2010-11-03,2011-09-09
nanoHUB_last_active_date__c,2010-11-03,2011-09-09


In [48]:
## filtering out accounts that have positive block attributes
spam_rows = df_sf[df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("1")]
good_rows = df_sf[~df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('1')]

## To Salesforce Sales Cloud CRM

In [49]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [50]:
display(spam_rows.head(2))
display(good_rows.tail(2))

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c,Organization_composite__c
356,å“æ“Ž,,ä½™,146690,yzqag,yzqag@pku.edu.cn,1,1,2016-06-17,2016-06-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/146690,,,a0r5w00000V42c6AAB,a0r5w00000V42c6AAB
364,å­Ÿ,,æ¢æ˜Š,182140,rubyrosa,rubyrosa@pku.edu.cn,1,1,2017-10-16,2017-10-16,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/182140,,,a0r5w00000V42c6AAB,a0r5w00000V42c6AAB


,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c,Organization_composite__c
186028,Gautam,,Khanna,302611,gautamk97,gautamk97@gmail.com,1,0,2020-10-08,2020-10-08,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/302611,,,NaN,NaN
186029,INGILELA,,ROHITH,302612,irohith,ee19btech11005@iith.ac.in,1,0,2020-10-08,2020-10-08,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/302612,,,NaN,NaN


In [51]:
# send data to Salesforce
db_s.send_data(good_rows)

[Success] Bulk job creation successful. Job ID = 7505w00000QhXKKAA3
hello
[Success] CSV upload successful. Job ID = 7505w00000QhXKKAA3
[Success] Closing job successful. Job ID = 7505w00000QhXKKAA3


In [57]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000QhXKKAA3',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-10-08T20:14:33.000+0000',
 'systemModstamp': '2020-10-08T20:24:39.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 177647,
 'numberRecordsFailed': 24,
 'retries': 0,
 'totalProcessingTime': 1569544,
 'apiActiveProcessingTime': 1355374,
 'apexProcessingTime': 14628587}

In [56]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


In [54]:
display(good_rows.tail(2))
print(good_rows.shape)

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c,Organization_composite__c
186028,Gautam,,Khanna,302611,gautamk97,gautamk97@gmail.com,1,0,2020-10-08,2020-10-08,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/302611,,,NaN,NaN
186029,INGILELA,,ROHITH,302612,irohith,ee19btech11005@iith.ac.in,1,0,2020-10-08,2020-10-08,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/302612,,,NaN,NaN


(177647, 16)


## Verification
Checking to see if salesforce has updated contacts with the new users

request name and nanohub_user_id